# KMeans

In [ ]:
from simpl_eeg import eeg_objects

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import patches
import seaborn as sns
import altair as alt

import mne
from mne.preprocessing import (create_eog_epochs, create_ecg_epochs,
                               compute_proj_ecg, compute_proj_eog)
import scipy.io
import scipy.interpolate
from scipy import signal
from scipy.cluster.hierarchy import (
    average,
    complete,
    dendrogram,
    fcluster,
    single,
    ward,
)

from sklearn import cluster, datasets, metrics
from sklearn.decomposition import PCA
from sklearn.datasets import make_blobs
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
from sklearn.datasets import make_moons

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import mglearn

from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# update figure size
plt.rcParams['figure.figsize'] = [18, 8]

# random state to make results reproducible
random_state = 42

## Selecting EEG data

We can select the data we want to use with the `simpl_eeg` package. We can either look at individual time steps or time steps averaged over time. We can also specify what time to look at.

For more information about making these selections, please see the page on [Creating EEG Objects](https://ubc-mds.github.io/simpl_eeg_capstone/eeg_objects.html) in the `simpl_eeg` documentation. 

Averaging the data has will reduce the dimensionality of the data, but which method you want to use will depend on what you are trying to achieve. 

In [ ]:
# experiment we want to use data from
experiment_number = "../../data/927"

# set the start second if you want to use a custom time
# when the start second is None, the impact times from the experiment will be used. 
start_second = 500
# start_second = None

# the number of seconds before the event to use
tmin = -5

# the number of seconds after the event to use
tmax = 5

epochs = eeg_objects.Epochs(experiment_number, tmin=tmin, tmax=tmax, start_second=start_second)
epoch = epochs.epoch

# the lines below are to average every n steps
# number of steps to average
n = 5

averaged_epoch = epochs.average_n_steps(n)

print("\nDimensionality difference between raw and averaged:")
print(f"Raw: {epoch.get_data().shape}")
print(f"Averaged: {averaged_epoch.get_data().shape}")

In [ ]:
# Choose which data you want by commenting or uncommenting the lines below:

# selected_var = epoch
selected_var = averaged_epoch

# Convert the data into a dataframe for easy analysis with clustering methods
df = selected_var.to_data_frame()

In [ ]:
df

In [ ]:
# Drop the time column and convert to array
df.drop(columns = ["time"], inplace=True)
X = df.to_numpy()

In [ ]:
X

## K-Means Clustering

 

**Clustering** is the task of partitioning the dataset into groups called clusters.

The goal of clustering is to discover underlying groups in a given dataset such that:
- examples in the same group are as similar as possible;
- examples in different groups are as different as possible.          

#### K-Means using `sklearn`

**Input**
- `X` $\rightarrow$ a set of data points  
- `K` (or $k$ or `n_clusters`) $\rightarrow$ number of clusters

**Output**
- `K` clusters (groups) of the data points (Represent each cluster by its cluster center and assign a cluster membership to each data point)

The number of clusters can be selected based on the methods for assessing goodness of fit in the [Quality Assessment](#Quality-Assessment) section below. 

In [ ]:
# Set parameters
n_clusters = 4

In [ ]:
# fit the data
kmeans_model = KMeans(n_clusters=n_clusters, random_state=random_state)
kmeans_model.fit(X)

In [ ]:
# predict the clusters
predictions = kmeans_model.predict(X)
predictions

In [ ]:
# view cluster centers
kmeans_model.cluster_centers_

In [ ]:
# Create new data frame with the predicted cluster assignment
df_predict = df.copy()
df_predict["Cluster"] = predictions
df_predict

## Quality Assessment

We used several methods for estimating the best value of the number of clusters (`K`) for the K-means algorithm. A summary of the results of each method is detailed below, with instructions on how to perform them. 

1)**[The Elbow Method](#The-Elbow-Method)** - determine best value for `K`

2)**[The Silhouette Method](#The-Silhouette-Method)** - alternative method to determine best value for `K`

3)**[Principal Component Analysis (PCA)](#Principal-Component-Analysis-(PCA))** - visualize clusters in 2D space to see if they are reasonably separated


### The Elbow Method

With the elbow method, you can set a range of values for `K` and visualize the result of the algorithm. 

- This method looks at the sum of **intra-cluster distances**, which is also referred to as **inertia**
- The inertia decreases as K increases
- The intra-cluster distance in our toy example above is given as   

$$ \sum_{P_i \in C_1}  distance(P_i, C_1)^2 + \sum_{P_i \in C_2}  distance(P_i, C_2)^2 + \sum_{P_i \in C_3} distance(P_i, C_3)^2$$

Where 
- $C_1, C_2, C_3$ are centroids 
- $P_i$s are points within that cluster
- $distance$ is the usual Euclidean distance

#### You can learn more about the Elbow method [here](https://www.scikit-yb.org/en/latest/api/cluster/elbow.html)

In [ ]:
# set the range of values of K to try
k_range = (1, 19)

model = KMeans(random_state=random_state)
visualizer = KElbowVisualizer(model, k=k_range)
visualizer.fit(X)
visualizer.show();

#### Results
We can see that there is an "elbow" starting to form at `K`=4, meaning that this method suggests 4 clusters is the ideal number. It also indicates that the inetria is sdropping at this level. 

Also, the algortithm computed the average score for all the clusters. 



### The Silhouette Method

- Not dependent on the notion of cluster centers 
- Calculated using the **mean intra-cluster distance** ($a$) and the **mean nearest-cluster distance** ($b$) for each sample
- the difference between the **the average nearest-cluster distance** ($b$) and **average intra-cluster distance** ($a$) for each sample, normalized by the maximum value

$$\frac{b-a}{max(a,b)}$$

- The best value is 1
- The worst value is -1 (samples have been assigned to wrong clusters)
- Value near 0 means overlapping clusters

The overall **Silhouette score** is the average of the Silhouette scores for all samples.

##### Interpretation

-	The plots show the Silhouette scores for each sample in that cluster
-	Higher scores indicate well separated clusters
-	The size represents the size of samples in each cluster
-	The thickness of each silhouette indicates the cluster size
-	The shape of each silhouette indicates the "goodness" for points in each cluster


#### You can learn more about the Silhouette Visualizer method [here](https://www.scikit-yb.org/en/latest/api/cluster/silhouette.html)

In [ ]:
# set k options to try
k_options = [3, 4, 5]

# store models for later use
models = dict()

for i in k_options:
    model = KMeans(i, random_state=random_state)
    visualizer = SilhouetteVisualizer(model, colors="yellowbrick")
    visualizer.fit(X)
    visualizer.show();
    models[i] = model # store the current model

#### Results

From the explained metrics above, we can say that k=3 is the optimal K value given the clusters shape size and the average Silhouette score. It is hard to interpret the plot in these methods. For instatnce the Elbow method indicated K=4 as an optimal while the Silhouette indicated K=3 is optimal in our opinion. 

### Principal Component Analysis (PCA)

In unsupervised learning techniques such as clustering are based on the notion of distances between points. With increased dimensions, the representation of data becomes more complex.

Dimensionality reduction is the task of reducing a dataset in high dimension (our df has many rows) to low dimension while retaining the most "important" characteristics of the data.

#### You can learn more about PCA [here](https://scikit-learn.org/stable/modules/decomposition.html#pca)

In [ ]:
def plot_pca_clusters(data, labels):
    """
    Carries out dimensionality reduction on the data for visualization
    """
    pca = PCA(n_components=2)
    principal_comp = pca.fit_transform(data)
    pca_df = pd.DataFrame(
        data=principal_comp, columns=["Principal Component Analysis (PCA)", ""], index=data.index
    )
    pca_df["cluster"] = labels
    plt.figure(figsize=(12, 6))
    ax = sns.scatterplot(
        x="Principal Component Analysis (PCA)", y="", hue="cluster", data=pca_df, palette="tab10"
    )
    plt.show()

for i, model in models.items():
    print(f"PCA for K={i}")
    plot_pca_clusters(df, model.labels_)

### Results
We can see that the clusters are not very well separated. This indicates that the clusters are not likely quite distinct, however it does not guarantee that the clusters does not represent something useful. In our case, it seems as though the clustering algorithm is picking up on high and low voltage values for creating the clusters. It makes sense that the clusters would look distinct as a result, but the finding is not significant for our goal of identifying brain states given that the voltage values are close to each other. 

## Density-Based Spatial Clustering of Applications with Noise (DBSCAN)
- DBSCAN is a density-based clustering algorithm
- Intuitively, it's based on the idea that clusters form dense regions in the data and so it works by identifying "crowded" regions in the feature space

In [ ]:
# Plot functions

def plot_X_dbscan(X, model):
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    colours = []
    if np.any(model.labels_ == -1):
        n_clusters = len(set(model.labels_)) - 1 
    else: 
        n_clusters = len(set(model.labels_))
    
    for i in range(n_clusters + 1):
        colours.append("#%06X" % np.random.randint(0, 0xFFFFFF))
        
    mglearn.discrete_scatter(X[:, 0], X[:, 1], ax=axes[0], markeredgewidth=1.0)

    if np.any(model.labels_ == -1):
        colours = ["w"] + colours
    mglearn.discrete_scatter(X[:, 0], X[:, 1], model.labels_, c=colours, markers="o", markeredgewidth=1.0, ax=axes[1]);
    plt.legend()
    
    
def plot_dbscan_with_labels(X, eps=1.0, min_samples = 2, font_size=14):
    model = DBSCAN(eps=eps, min_samples=min_samples) 
    model.fit(X)    
    if np.any(model.labels_ == -1):
        n_clusters = len(set(model.labels_)) - 1 
    else: 
        n_clusters = len(set(model.labels_))
    plt.title('Number of clusters: %d'%(n_clusters))
    colours = []
    for i in range(n_clusters + 1):
        colours.append("#%06X" % np.random.randint(0, 0xFFFFFF))
        
    #colours = [mglearn.cm3(1), mglearn.cm3(0)]
    if np.any(model.labels_ == -1):
        colours = ["w"] + colours
    mglearn.discrete_scatter(
        X[:, 0], X[:, 1], model.labels_, c=colours, markers="o", markeredgewidth=1.0
    );
    plt.legend()
    labels = [str(label) for label in list(range(0,len(X)))]
    for i, txt in enumerate(labels):
        plt.annotate(txt, X[i], xytext=X[i] + 0.2, size = font_size)


In [ ]:
pip install ipywidgets

In [ ]:
from ipywidgets import interactive

In [ ]:
# eps: determines what it means for points to be "close"
# min_samples: determines the number of neighboring points we require to consider in order for a point to be part of a cluster

dbscan = DBSCAN(eps=1.5, min_samples=2)
dbscan.fit(X)
interactive(lambda eps=1: plot_X_dbscan(X, dbscan), eps=(1, 50))
# plot_dbscan_with_labels(X, dbscan)
# interactive(lambda eps=1: plot_dbscan_with_labels(X, eps), eps=(1, 12, 1))

- Increasing `eps` ($\uparrow$) (left to right in the plot above) means more points will be included in a cluster.  
- Increasing `min_samples` ($\uparrow$) (top to bottom in the plot above) means points in less dense regions will either be labeled as their own cluster or noise. 
- In general, it's not trivial to tune these hyperparameters. 

There are three kinds of points:

- Core points are the points that have at least min_samples points in the neighborhood.

- Border points are the points with fewer than min_samples points in the neighborhood, but are connected to a core point.

- Noise points are the points which do not belong to any cluster. In other words, the points which have less that min_samples point within distance eps of the starting point are noise points.

Here we can see that DBSCAN has identified one cluster only; hence, the crowded region. It also identified all the points as noise! (If you try and change 'min_saples=2' to 'min_samples=1' you'll notice that each data point is in cluster). This is expected given the high dimensionality of our data. Although, DBSCAN did not identify any clusters, it's a very useful tool to use especially if you don't want to specify number of clusters (k). 

#### Let's evaluate different hyperparameters 

In [ ]:
dbscan = DBSCAN(eps=1.5, min_samples=2)
clusters = dbscan.fit_predict(X)

print("Cluster assignments:{}".format(clusters))

- noise points: shown in white
- core points: bigger
- border points: smaller

In [ ]:
mglearn.plots.plot_dbscan()

##### Here we can see after fitting X, that having min_samples = 2 and eps=1.5 will give us better clustering as there won't be any noises to be clustered.  

### K-Means vs. DBSCAN

- In DBSCAN, you do not have to specify the number of clusters! 
    - Instead, you have to tune `eps` and `min_samples`. 
- Unlike K-Means, DBSCAN doesn't have to assign all points to clusters. 
    - The label is -1 if a point is unassigned.
- Unlike K-Means, there is no `predict` method. 
    - DBSCAN only really clusters the points you have, not "new" or "test" points.

## Exploritory Data Analysis (EDA)


### Cluster Analysis and Interpretation

In [ ]:
# View number of values in each cluster

df_predict["Cluster"].value_counts()

In [ ]:
# View general statistics for each cluster

for i in range(len(df_predict["Cluster"].unique())):
    cluster_df = df_predict[df_predict["Cluster"] == i]
    print(f"\nCluster {i}")
    print(cluster_df.iloc[:, :5].describe()) # only view first 5 channels
    # print(cluster_df.describe()) # uncomment to view all channels

In [ ]:
# View all channels at once

plt.figure(figsize=(30, 30))

single_plot_df = df_predict.copy()
for i, col in enumerate(single_plot_df.columns):
    if col != "Cluster":
        single_plot_df[col] = single_plot_df[col] + 100*i
single_plot_df["time"] = single_plot_df.index
single_plot_df = single_plot_df.melt(id_vars=["Cluster", "time"])
ax = sns.scatterplot(
    x="time", y="value", hue="Cluster", data=single_plot_df, palette="tab10"
)
plt.show()

In [ ]:
# View channels in individual plots

plt.figure(figsize=(30, 6))

for col in df_predict.columns:
    if col != "Cluster":
        ax = sns.scatterplot(
            x=df_predict.index, y=col, hue="Cluster", data=df_predict, palette="tab10"
        )
        plt.show()

#### Results

We can clearly see that the K-means algorithm clustered the low/negative values in orange and the high/positve values in green. As for the middle cluster groups (green & blue), we need to invistigate further in order to come up with relationship and clear pattern. 

## Correlation between the channels 

In order to understand the correlation between the channels, we need to use statistics test that looks at the relationship between two continues variables and measure the linear correlation between them. A good test that fits here is The Pearson's Correlation Coefficient. It's a linear correlation coefficient that gives values between -1 and 1 where 1 indicates a strong positive correlation and -1 indicates a strong negative correlation. However, for this dataset we will use the Spearman's Rank-Order Correlation method, which is the nonparametric version of the Pearon's method. You may read more about the Spearman's method and reasoning behind using it [here](https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php#:~:text=The%20Spearman%27s%20rank%2Dorder%20correlation%20is%20the%20nonparametric%20version%20of,association%20between%20two%20ranked%20variables.).

In [ ]:
# Generate the heat correlation heatmap
plt.rcParams['figure.figsize'] = [20, 10]
sns.set(font_scale=1)
df_spearman = df.corr('spearman')
sns.heatmap(df_spearman, annot=True, cmap=plt.cm.Blues);

#### Using the Elbow method to determine the number of clusters for the spearman dataframe

In [ ]:
k_range = (3, 10)

model = KMeans(random_state=random_state)
visualizer = KElbowVisualizer(model, k=k_range)
visualizer.fit(df_spearman)
visualizer.show();

In [ ]:
# fit k=5 
kmeans_model = KMeans(n_clusters=5, random_state=random_state)
kmeans_model.fit(df_spearman)

In [ ]:
predictions2 = kmeans_model.predict(df_spearman)
predictions2

In [ ]:
# add Clusters column
df_spearman['Clusters'] = predictions2
df_spearman

In [ ]:
# Sort the df based on cluster values and drop the Clusters column
df_sort2 = df_spearman.sort_values('Clusters')
df_sort3 = df_sort2[list(df_sort2.index)]
df_sort3

In [ ]:
# Plot heatmap for the clustered dataframe
sns.set(font_scale=1)
ax = sns.heatmap(df_sort3.iloc[:,:-1], annot=True, cmap=plt.cm.Blues)
ax.hlines([8, 9, 14,16], color = 'limegreen', *ax.get_xlim(), linewidths=3) # numbers are coordinates for the axis
ax.vlines([8, 9, 14,16], color = 'limegreen', *ax.get_ylim(), linewidths=3) # numbers are coordinates for the axis

#### Results

Notice how Fp2 is noisy and suspecious hence it got clustered by itself. The green lines represent the boundaries between clusters. Also, we can see that nodes that are close to each other tend to be more correlated as seen by the darker blue sections along the diagonal of the heatmap and lighter blue near the outsides. This makes sense intuitively, because when a change in voltage occurs it may be picked up by multiple channels.

We can also examine the correlation of Fp2 using our topomap_2d function for further analysis:

In [ ]:
from simpl_eeg import topomap_2d, eeg_objects 

In [ ]:
topomap_2d.plot_topomap_2d(
        epoch,
        np.array(df_spearman['Fp2']),
        mark="channel_name",
        cmin=0,
        cmax=1,
        colormap="tab10",
        **{'image_interp':'none'}

    
    )

## Attribution

- Most of the material of this notebook came form DSCI_563 using the following license agreement

The following MIT License is applied to the code contained in this repository. The intent is for MDS students to be able to refer back to their course notes and reuse code for future projects. Students: note that the MIT License requires including the copyright/permission notice with the code.

MIT License

Copyright (c) 2021 Varada Kolhatkar, Rodolfo Lourenzutti, Mike Gelbart

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

https://github.ubc.ca/MDS-2020-21/DSCI_563_unsup-learn_students/blob/master/LICENSE.md